In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Load and Preprocess Datasets
def load_datasets():
    try:
        # File paths for the datasets
        crops_file = 'final_karnataka_dataset.csv'
        nutrients_file = 'nutrients.csv'
        age_nutrients_file = 'age_wise_nutrients_consumtion_final.csv'

        # Load datasets into DataFrames
        crops_df = pd.read_csv(crops_file)
        nutrients_df = pd.read_csv(nutrients_file)
        age_nutrients_df = pd.read_csv(age_nutrients_file)

        # Standardize column names and ensure lowercase for string content
        crops_df.columns = crops_df.columns.str.lower()
        nutrients_df.columns = nutrients_df.columns.str.lower()
        age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()

        crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

        print("Datasets loaded and preprocessed successfully!")
        return crops_df, nutrients_df, age_nutrients_df

    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None, None

# Step 2: Analyze Nutrient Deficiency for All Districts
def analyze_all_districts(crops_df, nutrients_df, age_nutrients_df):
    # Extract unique district names from crops dataset
    all_districts = crops_df['district_name'].str.lower().unique()
    all_results = {}

    for district_name in all_districts:
        print(f"\nProcessing District: {district_name.title()}")

        # Filter data for the selected district
        selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name]
        if selected_district_crops.empty:
            print(f"No crop data available for district '{district_name}'. Skipping...")
            continue

        # Merge crop production data with nutrient composition data
        district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
        nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                            'iron', 'calcium', 'potassium', 'magnesium']
        nutrient_totals = district_nutrient_data[nutrient_columns].sum()

        # Calculate deficiencies for all age groups
        deficiencies = {}
        for age_group in age_nutrients_df['group']:
            age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
            deficiency = age_rdi.values[0] - nutrient_totals.values
            deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
            deficiencies[age_group] = deficiency_dict

        # Identify the group with the least deficiency
        least_deficient_group = min(deficiencies, key=lambda g: sum(deficiencies[g].values()) if deficiencies[g] else float('inf'))
        if not deficiencies[least_deficient_group]:
            print(f"No deficiencies found for district '{district_name}'.")
            continue

        print(f"\nGroup with Least Deficiency: {least_deficient_group}")
        print("Deficient Nutrients:")
        for nutrient, amount in deficiencies[least_deficient_group].items():
            print(f"- {nutrient}: {amount:.2f}")

        # Project future deficiencies (10 years)
        growth_rate = 0.05  # 5% annual growth rate
        future_deficiencies = {nutrient: amount * (1 + growth_rate) ** 10
                                for nutrient, amount in deficiencies[least_deficient_group].items()}
        print("\n10-Year Projected Deficiencies:")
        for nutrient, amount in future_deficiencies.items():
            print(f"- {nutrient}: {amount:.2f}")

        # Recommend crops for balancing deficiencies
        print("\nSuggested Crops for Balancing Nutrients:")
        recommended_crops = {}
        for nutrient, deficit in deficiencies[least_deficient_group].items():
            nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
            top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')
            recommended_crops[nutrient] = top_crops
            print(f"- {nutrient}:")
            for crop in top_crops:
                print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")

        # Project production for the next 5 years
        print("\n5-Year Production Projections for Recommended Crops:")
        production_projections = {}
        for nutrient, crops in recommended_crops.items():
            for crop in crops:
                crop_name = crop['crop']
                crop_data = selected_district_crops[selected_district_crops['crop'].str.lower() == crop_name.lower()]
                if crop_data.empty:
                    continue

                yearly_production = crop_data.groupby('crop_year')['production'].sum().reset_index()
                if len(yearly_production) < 2:
                    print(f"Not enough data to project production for {crop_name}.")
                    continue

                # Linear regression for production projections
                X = yearly_production['crop_year'].values.reshape(-1, 1)
                y = yearly_production['production'].values
                model = LinearRegression()
                model.fit(X, y)

                future_years = np.arange(X.max() + 1, X.max() + 6).reshape(-1, 1)
                future_production = model.predict(future_years)
                production_projections[crop_name] = future_production.tolist()
                print(f"- {crop_name}: Future Production: {future_production.tolist()}")

        # Find neighboring districts with nutrient surplus
        print("\nNeighboring Districts with Surplus Nutrients:")
        surplus_nutrients = {}
        neighboring_districts = crops_df[crops_df['state_name'] == selected_district_crops['state_name'].iloc[0]]['district_name'].unique().tolist()
        neighboring_districts.remove(district_name)
        for nutrient in deficiencies[least_deficient_group]:
            surplus_nutrients[nutrient] = []
            for neighbor in neighboring_districts:
                neighbor_crops = crops_df[crops_df['district_name'].str.lower() == neighbor.lower()]
                neighbor_data = neighbor_crops.merge(nutrients_df, on='crop', how='inner')
                neighbor_surplus = neighbor_data[nutrient].sum()

                if neighbor_surplus > 0:
                    surplus_nutrients[nutrient].append({
                        'district': neighbor,
                        'crops': neighbor_data[['crop', nutrient]].to_dict(orient='records'),
                        'surplus': neighbor_surplus
                    })
        for nutrient, districts in surplus_nutrients.items():
            print(f"- {nutrient}:")
            for district in districts:
                print(f"  District: {district['district']}, Surplus: {district['surplus']}, Crops: {district['crops']}")

        # Save results for the district
        all_results[district_name] = {
            "least_deficient_group": least_deficient_group,
            "deficiencies": deficiencies[least_deficient_group],
            "future_deficiencies": future_deficiencies,
            "recommended_crops": recommended_crops,
            "production_projections": production_projections,
            "surplus_nutrients": surplus_nutrients
        }

    return all_results

# Load datasets
crops_df, nutrients_df, age_nutrients_df = load_datasets()

# Perform analysis
if crops_df is not None and nutrients_df is not None and age_nutrients_df is not None:
    results = analyze_all_districts(crops_df, nutrients_df, age_nutrients_df)

    # Display results
    for district, data in results.items():
        print(f"\n=== District: {district.title()} ===")
        print(f"Group with Least Deficiency: {data['least_deficient_group']}")
        print("Deficient Nutrients:")
        for nutrient, amount in data['deficiencies'].items():
            print(f"- {nutrient}: {amount:.2f}")
        print("\n10-Year Projected Deficiencies:")
        for nutrient, amount in data['future_deficiencies'].items():
            print(f"- {nutrient}: {amount:.2f}")
        print("\nSuggested Crops for Balancing Nutrients:")
        for nutrient, crops in data['recommended_crops'].items():
            print(f"- {nutrient}:")
            for crop in crops:
                print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
        print("\n5-Year Production Projections:")
        for crop, future in data['production_projections'].items():
            print(f"- {crop}: {future}")
        print("\nNeighboring Districts with Surplus Nutrients:")
        for nutrient, surplus in data['surplus_nutrients'].items():
            print(f"- {nutrient}:")


/tmp/ipykernel_449831/3476507915.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_449831/3476507915.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_449831/3476507915.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Datasets loaded and preprocessed successfully!

Processing District: Belgaum

Group with Least Deficiency: adult female crossbred (1000 number)
Deficient Nutrients:
- carbohydrates: 109469395.80
- protein: 16830876.60
- fiber: 9745194.80
- fat: 25539162.30
- vitamin a: 255185769.00
- vitamin c: 27369874.70
- iron: 6567231.50
- calcium: 364999863.20
- potassium: 1715154198.00
- magnesium: 113054461.00

10-Year Projected Deficiencies:
- carbohydrates: 178314110.62
- protein: 27415724.46
- fiber: 15873895.45
- fat: 41600604.24
- vitamin a: 415670727.95
- vitamin c: 44582641.83
- iron: 10697328.10
- calcium: 594546315.94
- potassium: 2793805457.22
- magnesium: 184153804.06

Suggested Crops for Balancing Nutrients:
- carbohydrates:
  * other fibres (Nutrient Value: 80.0)
  * ginger (Nutrient Value: 80.0)
  * jute (Nutrient Value: 76.0)
- protein:
  * soyabean (Nutrient Value: 36.0)
  * rapeseed & mustard (Nutrient Value: 26.0)
  * groundnut (Nutrient Value: 25.8)
- fiber:
  * cardamom (Nutr

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

